# Scraping API Yelp: Bodega Biarritz 1881

__________________________________________________________________________________________________________________

### Objective

1. Get main information of Bodega Biarritz 1881
2. Is Yelp having the same type of reviews that we found in TripAdvisor?

### Web Scraping Steps (using API)
0. Imports
1. Get main restaurant info including all possible reviews 
2. Analyse all reviews
    - Sentiment Analysis
    - Evolution 
    - Most mention words     
3. Get Conclusions
__________________________________________________________________________________________________________________

#### 0. import

In [576]:
import requests
import pandas as pd
from textblob import TextBlob
import pprint
import re

#### 1. Get main restaurant info including all possible reviews 

*Steps to get to reviews of a particular restaurant, in our case:Bodega Biarritz 1881*

1. Find the restaurant ID 

    I'll do it throught the yelp api "https://api.yelp.com/v3/businesses/search/phone" as we can look for its phone in internet and we suposu only this restaurant will have this unique phone number
    
    
2. With the restaurant ID find all its reviews

In [577]:
#FIND ID RESTAURANT

api_key="vbaLOLzKsvQuf_ZnqDsWeGtWS5nzU8lDkj4gH0LDQ7jF7sVhS3IGBUVohAkDKLKUBOawngNwQ5qUcONmHxbkbLJXjSi2i6yzRhahCVtykCHSEaKO-5urVdU8hLY_X3Yx"
url="https://api.yelp.com/v3/businesses/search/phone"
headers={"Authorization":"Bearer %s" %api_key}
params={'phone': '+34618677927'}

In [578]:
response=requests.get(url, params=params, headers=headers)
response.status_code

200

In [579]:
data=response.json()

In [580]:
#Flattering data
flattened_data=pd.json_normalize(data)

In [581]:
flattened_data

,businesses,total
0,"[{'id': 'qvAtonLNW9hT3zkE-8zF2A', 'alias': 'bo...",2


In [582]:
#As we see, we have all info in 1 column. 
#We will divided nicely.
data_restaurant=flattened_data["businesses"][0]
data_restaurant=pd.json_normalize(data_restaurant)

In [583]:
#Here we have all the main info about the restaurant.
#We'll take the first ID restaurant as the referent becuase it is the same address we had review for TripAdvisor. 
data_restaurant


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,...,coordinates.latitude,coordinates.longitude,location.address1,location.address2,location.address3,location.city,location.zip_code,location.country,location.state,location.display_address
0,qvAtonLNW9hT3zkE-8zF2A,bodega-biarritz-1881-barcelona,Bodega Biarritz 1881,https://s3-media0.fl.yelpcdn.com/bphoto/zpE0zp...,False,https://www.yelp.com/biz/bodega-biarritz-1881-...,90,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,[],...,41.379771,2.176196,"Carrer del Vidre, 8",,,Barcelona,08002,ES,B,"[Carrer del Vidre, 8, 08002 Barcelona, Spain]"
1,5IrOyJvyCA4IImq08n_sDQ,bodega-biarritz-barcelona,Bodega Biarritz,https://s3-media0.fl.yelpcdn.com/bphoto/Dem2QU...,False,https://www.yelp.com/biz/bodega-biarritz-barce...,6,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.5,[],...,41.379240,2.177010,"Carrer Nou De Sant Francesc, 7",,None,Barcelona,08002,ES,B,"[Carrer Nou De Sant Francesc, 7, 08002 Barcelo..."


In [584]:
#Save data to csv
data_restaurant.to_csv('yelp_data_restaurant_BodegaBiarritz.csv')

In [689]:
#Open csv data_restaurant to clean table
data_restaurant=pd.read_csv("yelp_data_restaurant_BodegaBiarritz.csv")

In [690]:
data_restaurant.columns

Index(['Unnamed: 0', 'id', 'alias', 'name', 'image_url', 'is_closed', 'url',
       'review_count', 'categories', 'rating', 'transactions', 'price',
       'phone', 'display_phone', 'coordinates.latitude',
       'coordinates.longitude', 'location.address1', 'location.address2',
       'location.address3', 'location.city', 'location.zip_code',
       'location.country', 'location.state', 'location.display_address'],
      dtype='object')

In [691]:
data_restaurant.drop(columns=['Unnamed: 0','alias','display_phone','transactions','price', 'image_url','is_closed','location.address2',
       'location.address3','location.display_address'], inplace=True)

In [692]:
data_restaurant

,id,name,url,review_count,categories,rating,phone,coordinates.latitude,coordinates.longitude,location.address1,location.city,location.zip_code,location.country,location.state
0,qvAtonLNW9hT3zkE-8zF2A,Bodega Biarritz 1881,https://www.yelp.com/biz/bodega-biarritz-1881-...,90,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,34618677927,41.379771,2.176196,"Carrer del Vidre, 8",Barcelona,8002,ES,B
1,5IrOyJvyCA4IImq08n_sDQ,Bodega Biarritz,https://www.yelp.com/biz/bodega-biarritz-barce...,6,"[{'alias': 'tapasmallplates', 'title': 'Tapas/...",4.5,34618677927,41.379240,2.177010,"Carrer Nou De Sant Francesc, 7",Barcelona,8002,ES,B


In [693]:
#Save clean data_restaurant 
data_restaurant.to_csv("yelp_data_restaurant_BodegaBiarritz_Clean.csv")

In [586]:
#FIND REVIEWS RESTAURANT (ID:qvAtonLNW9hT3zkE-8zF2A->we use this one)

#Get Key, Url + Headers
id_restaurant="qvAtonLNW9hT3zkE-8zF2A"
api_key="vbaLOLzKsvQuf_ZnqDsWeGtWS5nzU8lDkj4gH0LDQ7jF7sVhS3IGBUVohAkDKLKUBOawngNwQ5qUcONmHxbkbLJXjSi2i6yzRhahCVtykCHSEaKO-5urVdU8hLY_X3Yx"
url=f"https://api.yelp.com/v3/businesses/{id_restaurant}/reviews"
headers={"Authorization":"Bearer %s" %api_key}



In [587]:
#Get the info from url
response=requests.get(url,headers=headers)

In [588]:
response

<Response [200]>

In [589]:
#Get response with json format
data=response.json()

In [590]:
data

{'reviews': [{'id': 'QvRE0hOZb8hLHXNR1KLaJw',
   'url': 'https://www.yelp.com/biz/bodega-biarritz-1881-barcelona?adjust_creative=56tMv8Oeq8QD6pC-p_IovA&hrid=QvRE0hOZb8hLHXNR1KLaJw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=56tMv8Oeq8QD6pC-p_IovA',
   'text': 'Was sitting in Plaça Real looking at restaurant\nreviews and saw this.  Wow wow wow.  What a find.  Delicious food.  Reasonable prices. Staff that are...',
   'rating': 5,
   'time_created': '2020-03-12 14:32:09',
   'user': {'id': 'VkaQuKn57hR0lEOQhlf0cg',
    'profile_url': 'https://www.yelp.com/user_details?userid=VkaQuKn57hR0lEOQhlf0cg',
    'image_url': None,
    'name': 'George T.'}},
  {'id': 'uthsxDzJd2oos-3j1NfeDg',
   'url': 'https://www.yelp.com/biz/bodega-biarritz-1881-barcelona?adjust_creative=56tMv8Oeq8QD6pC-p_IovA&hrid=uthsxDzJd2oos-3j1NfeDg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=56tMv8Oeq8QD6pC-p_IovA',
   'text': 'Space is really limited, and the interior

In [591]:
#As we see, we have all info in 1 column again. We will divided nicely.

flattened_data=pd.json_normalize(data)
flattened_data

,reviews,total,possible_languages
0,"[{'id': 'QvRE0hOZb8hLHXNR1KLaJw', 'url': 'http...",90,"[fr, de, en]"


In [592]:
#Here we have all the info from 3 reviews out of 90 reviews
reviews=flattened_data["reviews"][0]

In [593]:
reviews=pd.json_normalize(reviews)
reviews

,id,url,text,rating,time_created,user.id,user.profile_url,user.image_url,user.name
0,QvRE0hOZb8hLHXNR1KLaJw,https://www.yelp.com/biz/bodega-biarritz-1881-...,Was sitting in Plaça Real looking at restauran...,5,2020-03-12 14:32:09,VkaQuKn57hR0lEOQhlf0cg,https://www.yelp.com/user_details?userid=VkaQu...,None,George T.
1,uthsxDzJd2oos-3j1NfeDg,https://www.yelp.com/biz/bodega-biarritz-1881-...,"Space is really limited, and the interior gets...",5,2019-11-23 08:36:55,lGBvmBDeIooqcrMwp1a1yg,https://www.yelp.com/user_details?userid=lGBvm...,https://s3-media1.fl.yelpcdn.com/photo/2AcVoHA...,Eugene W.
2,TWMRgvXS3Omyj52AN-xLew,https://www.yelp.com/biz/bodega-biarritz-1881-...,Came around 6:30-7pm during the week and waite...,5,2019-10-02 07:59:00,f2-ZHyhJ025sJSrFHeUTZA,https://www.yelp.com/user_details?userid=f2-ZH...,https://s3-media4.fl.yelpcdn.com/photo/0OuQypK...,Nancy B.


In [594]:
#Save data to csv
reviews.to_csv('yelp_reviews_restaurant_BodegaBiarritz.csv')

#### 2. Analyse all reviews

##### Cleaning columns: delet usless columns + rewrite name columns + clean time column

In [628]:
reviews=pd.read_csv("yelp_reviews_restaurant_BodegaBiarritz.csv")

In [629]:
reviews

,Unnamed: 0,id,url,text,rating,time_created,user.id,user.profile_url,user.image_url,user.name
0,0,QvRE0hOZb8hLHXNR1KLaJw,https://www.yelp.com/biz/bodega-biarritz-1881-...,Was sitting in Plaça Real looking at restauran...,5,2020-03-12 14:32:09,VkaQuKn57hR0lEOQhlf0cg,https://www.yelp.com/user_details?userid=VkaQu...,NaN,George T.
1,1,uthsxDzJd2oos-3j1NfeDg,https://www.yelp.com/biz/bodega-biarritz-1881-...,"Space is really limited, and the interior gets...",5,2019-11-23 08:36:55,lGBvmBDeIooqcrMwp1a1yg,https://www.yelp.com/user_details?userid=lGBvm...,https://s3-media1.fl.yelpcdn.com/photo/2AcVoHA...,Eugene W.
2,2,TWMRgvXS3Omyj52AN-xLew,https://www.yelp.com/biz/bodega-biarritz-1881-...,Came around 6:30-7pm during the week and waite...,5,2019-10-02 07:59:00,f2-ZHyhJ025sJSrFHeUTZA,https://www.yelp.com/user_details?userid=f2-ZH...,https://s3-media4.fl.yelpcdn.com/photo/0OuQypK...,Nancy B.


In [630]:

#delet usless columns
reviews.drop(columns=['id', 'url', 'user.id',
       'user.profile_url', 'user.image_url', 'user.name'], inplace=True)

In [631]:
reviews

,Unnamed: 0,text,rating,time_created
0,0,Was sitting in Plaça Real looking at restauran...,5,2020-03-12 14:32:09
1,1,"Space is really limited, and the interior gets...",5,2019-11-23 08:36:55
2,2,Came around 6:30-7pm during the week and waite...,5,2019-10-02 07:59:00


In [632]:
reviews.drop(columns=["Unnamed: 0"],  inplace=True)

In [633]:
#rewrite name columns
reviews.rename(columns={"text":"Full Review"}, inplace=True)

In [634]:
reviews.rename(columns={"rating":"Rating"}, inplace=True)

In [635]:
reviews.rename(columns={"time_created":"Date of Review"}, inplace=True)

In [636]:
#check results
reviews

,Full Review,Rating,Date of Review
0,Was sitting in Plaça Real looking at restauran...,5,2020-03-12 14:32:09
1,"Space is really limited, and the interior gets...",5,2019-11-23 08:36:55
2,Came around 6:30-7pm during the week and waite...,5,2019-10-02 07:59:00


In [637]:
#clean time column
reviews["Date of Review"]=reviews["Date of Review"].str.replace("(\d{2}:\d{2}:\d{2})","")
reviews["Date of Review"]

0    2020-03-12 
1    2019-11-23 
2    2019-10-02 
Name: Date of Review, dtype: object

In [638]:
#comprobamos
reviews

,Full Review,Rating,Date of Review
0,Was sitting in Plaça Real looking at restauran...,5,2020-03-12
1,"Space is really limited, and the interior gets...",5,2019-11-23
2,Came around 6:30-7pm during the week and waite...,5,2019-10-02


In [639]:
#Split year,month and date 
reviews["Date of Review"]=reviews["Date of Review"].str.split("-")


In [640]:
reviews["Date of Review"][0][0]

'2020'

In [641]:
reviews["Year of Review"]=[i[0] for i in reviews["Date of Review"]]
reviews["Year of Review"]


0    2020
1    2019
2    2019
Name: Year of Review, dtype: object

In [642]:
reviews["Month of Review"]=[i[1] for i in reviews["Date of Review"]]
reviews["Month of Review"]

0    03
1    11
2    10
Name: Month of Review, dtype: object

In [643]:
reviews["Day of Review"]=[i[2] for i in reviews["Date of Review"]]
reviews["Day of Review"]

0    12 
1    23 
2    02 
Name: Day of Review, dtype: object

In [644]:
#comprobamos
reviews

,Full Review,Rating,Date of Review,Year of Review,Month of Review,Day of Review
0,Was sitting in Plaça Real looking at restauran...,5,"[2020, 03, 12 ]",2020,03,12
1,"Space is really limited, and the interior gets...",5,"[2019, 11, 23 ]",2019,11,23
2,Came around 6:30-7pm during the week and waite...,5,"[2019, 10, 02 ]",2019,10,02


##### Adding columns with usefull data

In [645]:
#añadimos columna con el total de palabras que tiene cada review.
#hypothesis: review más largas nos dan más info
reviews["Total Words Review"]=reviews["Full Review"].apply(lambda x: len(x.split()))

In [646]:
#comprobamos
reviews

,Full Review,Rating,Date of Review,Year of Review,Month of Review,Day of Review,Total Words Review
0,Was sitting in Plaça Real looking at restauran...,5,"[2020, 03, 12 ]",2020,03,12,25
1,"Space is really limited, and the interior gets...",5,"[2019, 11, 23 ]",2019,11,23,28
2,Came around 6:30-7pm during the week and waite...,5,"[2019, 10, 02 ]",2019,10,02,29


In [647]:
#Adding stopwords of each review
import nltk
from nltk.corpus import stopwords 
stop_words=stopwords.words("english")

In [648]:
reviews["Total StopWords"]=reviews["Full Review"].apply(lambda x: len([word for word in x.split() if word.lower() in stop_words]))


In [649]:
#comprobamos
reviews

,Full Review,Rating,Date of Review,Year of Review,Month of Review,Day of Review,Total Words Review,Total StopWords
0,Was sitting in Plaça Real looking at restauran...,5,"[2020, 03, 12 ]",2020,03,12,25,7
1,"Space is really limited, and the interior gets...",5,"[2019, 11, 23 ]",2019,11,23,28,11
2,Came around 6:30-7pm during the week and waite...,5,"[2019, 10, 02 ]",2019,10,02,29,14


In [650]:
#adding clean "Full Review" column to analyse it later

#first we turn all words to lower case words
reviews["Clean Full Review"]=reviews["Full Review"].apply(lambda x: " ".join(word.lower() for word in x.split()))


In [651]:
#second we remove punctuation
reviews["Clean Full Review"]=reviews["Clean Full Review"].str.replace("[^\w\s]","")


In [652]:
#third we take out StopeWords
reviews["Clean Full Review"]=reviews["Clean Full Review"].apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))


In [653]:
#comprobamos
reviews

,Full Review,Rating,Date of Review,Year of Review,Month of Review,Day of Review,Total Words Review,Total StopWords,Clean Full Review
0,Was sitting in Plaça Real looking at restauran...,5,"[2020, 03, 12 ]",2020,03,12,25,7,sitting plaça real looking restaurant reviews ...
1,"Space is really limited, and the interior gets...",5,"[2019, 11, 23 ]",2019,11,23,28,11,space really limited interior gets pretty pack...
2,Came around 6:30-7pm during the week and waite...,5,"[2019, 10, 02 ]",2019,10,02,29,14,came around 6307pm week waited 30 minutes tabl...


##### Reorganize data to have a nice table

In [654]:
#Set "Year of Review" as Index follow by "Month of review" , "Day of review" and rating 
reviews=reviews.sort_values(["Year of Review","Month of Review","Day of Review","Rating"], ascending=[True,True,True,True])


In [655]:
reviews.columns

Index(['Full Review', 'Rating', 'Date of Review', 'Year of Review',
       'Month of Review', 'Day of Review', 'Total Words Review',
       'Total StopWords', 'Clean Full Review'],
      dtype='object')

In [659]:
#Chose columns to be in thee final table and inthe right order
reviews=reviews[['Year of Review','Month of Review', 'Day of Review','Rating','Full Review','Clean Full Review', 'Total Words Review','Total StopWords']]
reviews

,Year of Review,Month of Review,Day of Review,Rating,Full Review,Clean Full Review,Total Words Review,Total StopWords
2,2019,10,02,5,Came around 6:30-7pm during the week and waite...,came around 6307pm week waited 30 minutes tabl...,29,14
1,2019,11,23,5,"Space is really limited, and the interior gets...",space really limited interior gets pretty pack...,28,11
0,2020,03,12,5,Was sitting in Plaça Real looking at restauran...,sitting plaça real looking restaurant reviews ...,25,7


##### Sentiment Analysis
Althoug we see that 3 reviews are too little to analyse, we'll do it to follow tripadvysors same analysis.

In [660]:
reviews["Clean Full Review"].apply(lambda x: TextBlob(x).sentiment)


2                                  (0.8, 0.75)
1    (0.16964285714285715, 0.5204365079365079)
0     (0.2833333333333333, 0.8166666666666665)
Name: Clean Full Review, dtype: object

In [661]:
#Creamos la columna para la Polarity Rate 
reviews["Polarity Rate"]=reviews["Clean Full Review"].apply(lambda x: TextBlob(x).sentiment[0])


In [662]:
#Creamos la columna para la Subjectivity Rate 
reviews["Subjectivity Rate"]=reviews["Clean Full Review"].apply(lambda x: TextBlob(x).sentiment[1])


In [663]:
#comprobamos
reviews

,Year of Review,Month of Review,Day of Review,Rating,Full Review,Clean Full Review,Total Words Review,Total StopWords,Polarity Rate,Subjectivity Rate
2,2019,10,02,5,Came around 6:30-7pm during the week and waite...,came around 6307pm week waited 30 minutes tabl...,29,14,0.800000,0.750000
1,2019,11,23,5,"Space is really limited, and the interior gets...",space really limited interior gets pretty pack...,28,11,0.169643,0.520437
0,2020,03,12,5,Was sitting in Plaça Real looking at restauran...,sitting plaça real looking restaurant reviews ...,25,7,0.283333,0.816667


In [666]:
#Save clean data to csv
reviews.to_csv('yelp_reviews_restaurant_BodegaBiarritz_Clean.csv')

#### 3. Get Conclusions

- No conclusions can be taken regarding data as it is too little to trust.
- Scrpaing websites it gives more freedom and extendet data than using an API (compared to  Tripadvisor review project)